# Applied Data Science Capstone Week 3 Assignment

### 1. Start by creating a new Notebook for this assignment.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

#had to install lxml with !conda install -c anaconda lxml --yes

Libraries imported.


### 2. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [2]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### 3. To create the above dataframe:

<ul>
<li>The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood</li>
<li>Only process the cells that have an assigned borough. Ignore cells with a borough that is <b>Not assigned</b>.</li>
</ul>

In [3]:
df=df.loc[df['Borough'] != 'Not assigned']
df=df.reset_index(drop=True)

In [4]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<ul><li>More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that <b>M5A</b> is listed twice and has two neighborhoods: <b>Harbourfront</b> and <b>Regent Park</b>. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in <b>row 11</b> in the above table.</li></ul>

In [5]:
df1=df.groupby('Postcode')['Neighbourhood'].apply(', '.join)
df=df.drop('Neighbourhood', 1)
df=pd.merge(df, df1, on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"


In [6]:
df=df.drop_duplicates(keep='last').reset_index(drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


<ul>
<li>If a cell has a borough but a <b>Not assigned</b> neighborhood, then the neighborhood will be the same as the borough. So for the <b>9th</b> cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be <b>Queen's Park</b>.</li>
</ul>

In [7]:
df.loc[df['Neighbourhood']=='Not assigned', 'Neighbourhood']=df['Borough']

In [8]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


<ul>
<li>Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.</li>
<li>In the last cell of your notebook, use the <b>.shape</b> method to print the number of rows of your dataframe.</li>
</ul>

In [9]:
df.shape

(103, 3)

# Part 2
#### downloading coordinates from a csv file at http://cocl.us/Geospatial_data and loading into a different dataframe

In [27]:
df_coordinates=pd.read_csv('https://cocl.us/Geospatial_data')

In [28]:
#Renaming the columns names
df_coordinates=df_coordinates.rename(columns = {"Postal Code": "PostalCode"}) 
df=df.rename(columns = {"Postcode": "PostalCode"}) 

In [29]:
#Merging two dataframes bases on same column i.e. PostalCode
df=pd.merge(df, df_coordinates, on='PostalCode')

In [31]:
df.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


# Part3

In [35]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [38]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add Borough as markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto